#### Riley's new gene project has shown that when a GFP gene is induced in the e.coli, there is a consistent shift of proteome fraction of essential proteins. i.e. most essential proteins have like 10% smaller fraction compared to wild type e.coli. The purpose of this notebook is to identify any connection between the decrease in expression of these essential proteins and potential metabolic defect. For this purpose, we will 1). import the complex ids of essential proteins that have decreased proteome fraction, 2). identify the kinetic reactions catalyzed by some of these complexes, 3). perform FBA to see if there is any metabolic defect when these reactions are knocked downed (80% for instance).

In [1]:
import pandas as pd
import os, dill
import numpy as np
import cvxpy as cp
from typing import Iterable, Optional, Mapping, cast
from plotly.graph_objects import Scatter, Figure

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
os.chdir(os.path.expanduser('~/dev/vEcoli'))

%load_ext autoreload

In [2]:
# import complex id of interest from txt
complex_ids_txt = pd.read_csv('notebooks/Heena notebooks/Riley New Genes/proteome_fraction_scatterplot_below_line_essential_complex_ids_16.txt', header=None)
complex_ids = np.unique(complex_ids_txt[0].tolist())

# import kinetic reaction info from sim result
time = '400'
date = '2025-05-15'
experiment = 'NEW_NewGenes_checkpoint2'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/metabolism-comparison/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
output = output['agents']['0']
fba = output['listeners']['fba_results']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

In [3]:
# get commonly stored variables
metabolism = agent['ecoli-metabolism-redux-classic']
stoichiometry = metabolism.stoichiometry.copy()
reaction_names = metabolism.reaction_names
fba_new_reaction_ids = metabolism.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids = metabolism.parameters['fba_reaction_ids_to_base_reaction_ids']
metabolites = metabolism.metabolite_names.copy()
binary_kinetic_idx = metabolism.binary_kinetic_idx
exchange_molecules = metabolism.exchange_molecules
kinetic_enzymes = metabolism.parameters['kinetic_constraint_enzymes']

S = stoichiometry .copy()
S = pd.DataFrame(S, index=metabolites , columns=reaction_names )
homeostatic_count = pd.DataFrame(fba["homeostatic_metabolite_counts"], columns=metabolism.homeostatic_metabolites).mean(axis=0)
homeostatic = pd.DataFrame(fba["target_homeostatic_dmdt"], columns=metabolism.homeostatic_metabolites).mean(axis=0)
maintenance = pd.DataFrame(fba["maintenance_target"][1:], columns=['maintenance_reaction']).mean(axis=0)
kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).mean(axis=0).copy()

kinetic_reaction_ids = metabolism.kinetic_constraint_reactions
allowed_exchange_uptake = metabolism.allowed_exchange_uptake
FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]

In [4]:
# get overlap between kinetic catalyst and complexes of interest
kinetic_catalyst_overlapped = np.intersect1d(kinetic_enzymes, complex_ids)
print(f'There are {len(kinetic_catalyst_overlapped)} complexes of interest that are also kinetic catalysts.')

There are 54 complexes of interest that are also kinetic catalysts.


In [5]:
# link them to kinetic reactions
kinetic_reaction_catalysts = {key:metabolism.parameters['reaction_catalysts'].get(key) for key in kinetic_reaction_ids}
kinetic_reaction_catalysts_essential = {key:value for key, value in kinetic_reaction_catalysts.items() if np.any(np.isin(value, kinetic_catalyst_overlapped))}

## Test FBA with decreased kinetic target for these reactions

In [19]:
kinetic_reaction_id_essential = kinetic_reaction_catalysts_essential.keys()
kinetic_target_essential = kinetic.loc[kinetic_reaction_id_essential]


# all reactions
sim = pd.DataFrame(fba["estimated_fluxes"], columns= reaction_names).mean(axis=0).copy()

# kinetic reactions
kc_target = pd.DataFrame(fba["target_kinetic_fluxes"], columns= kinetic_reaction_ids).mean(axis=0).copy()

# get kinetic_reaction_idx
kinetic_reaction_idx = np.array([reaction_names.index(id) for id in kinetic_reaction_ids])

# form idx, flux dict
WT_fluxes = sim.iloc[kinetic_reaction_idx]

kinetic_constraint_dict = dict(zip(kinetic_reaction_idx,WT_fluxes))

{np.int64(9): 0.0,
 np.int64(10): 0.0,
 np.int64(12): 0.0,
 np.int64(80): 0.0,
 np.int64(81): 0.0,
 np.int64(82): 0.0,
 np.int64(83): 0.0,
 np.int64(84): 0.0,
 np.int64(85): 0.0,
 np.int64(88): 687.035,
 np.int64(100): 14720.5875,
 np.int64(103): 27.7825,
 np.int64(117): 371.1275,
 np.int64(121): 2.8025,
 np.int64(122): 0.0,
 np.int64(135): 14722.4775,
 np.int64(143): 0.0,
 np.int64(144): 0.0,
 np.int64(162): 0.0,
 np.int64(164): 0.0,
 np.int64(165): 0.0,
 np.int64(166): 0.0,
 np.int64(185): 0.0,
 np.int64(204): 0.0,
 np.int64(206): 0.0,
 np.int64(262): 9541.645,
 np.int64(267): 0.0,
 np.int64(276): 0.0,
 np.int64(282): 0.0,
 np.int64(312): 318638.4875,
 np.int64(313): 0.0,
 np.int64(315): 0.0,
 np.int64(542): 0.0,
 np.int64(550): 45707.3925,
 np.int64(551): 0.0,
 np.int64(554): 0.0,
 np.int64(559): 14341.5375,
 np.int64(605): 0.1975,
 np.int64(608): 86.15,
 np.int64(611): 0.0,
 np.int64(615): 2.125,
 np.int64(616): 17669.57,
 np.int64(617): 9541.645,
 np.int64(635): 28.0975,
 np.int64

In [20]:
def get_subset_S(S, met_of_interest):
    S_met = S.loc[met_of_interest, :]
    S_met = S_met.loc[:,~np.all(S_met == 0, axis=0)]
    return S_met, S_met.columns

def get_keys(dict, value):
    return [key for key in dict if np.any(np.isin(value, dict[key]))]

def test_NetworkFlowModel(objective_weights,
                          uptake_addition = set([]), uptake_removal = set([]), new_exchange_molecules = set([]),
                          add_metabolite = None, add_reaction = None, add_kinetic = None, remove_reaction = None, force_reaction = None,
                          add_homeostatic_demand = None, solver_choice=cp.GLOP):
    # update exchanges
    uptake = metabolism.allowed_exchange_uptake.copy()
    uptake = set(uptake)
    uptake = uptake | uptake_addition
    uptake = uptake - uptake_removal

    exchange_molecules = metabolism.exchange_molecules.copy()
    exchange_molecules = exchange_molecules | new_exchange_molecules

    # update stoichiometry
    reaction_names = metabolism.reaction_names.copy()
    kinetic_reaction_ids = metabolism.kinetic_constraint_reactions.copy()
    kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).loc[24, :].copy()
    metabolites = metabolism.metabolite_names.copy()
    homeostatic_counts = homeostatic_count.copy() * metabolism.counts_to_molar.asNumber()

    S_new = stoichiometry.copy()

    if add_metabolite is not None: # add to metabolites list because they are currently not included in the model
        for m in add_metabolite:
            if m not in metabolites:
                metabolites.append(m)
        # append rows of zeros to S_new of length add_metabolite
        S_new = np.concatenate((S_new, np.zeros((len(add_metabolite), S_new.shape[1]))), axis=0)

    if add_reaction is not None:
        # assert add_reaction is a dictionary
        assert isinstance(add_reaction, dict)

        for r,s in add_reaction.items():
            if r not in reaction_names:
                reaction_names.append(r)
            # append columns of reaction stoich to S_new of length add_reaction
            new_reaction = np.zeros((S_new.shape[0], 1))
            for m, v in s.items():
                new_reaction[metabolites.index(m), 0] = v
            S_new = np.concatenate((S_new, new_reaction), axis=1)

    if add_kinetic is not None:
        # assert add_kinetic is a dictionary
        assert isinstance(add_kinetic, dict)

        for r, v in add_kinetic.items():
            if r not in kinetic_reaction_ids:
                kinetic_reaction_ids.append(r)
                kinetic[r] = v
            if r in kinetic_reaction_ids:
                kinetic[r] = v

    if remove_reaction is not None:
        for r in remove_reaction:
            r_idx = reaction_names.index(r)
            S_new = np.delete(S_new, r_idx, axis=1)
            reaction_names.remove(r)
            if r in kinetic_reaction_ids:
                kinetic_reaction_ids.remove(r)
                del kinetic[r]

    if force_reaction is not None:
        force_reaction_idx = np.array([reaction_names.index(r) for r in force_reaction])
    else:
        force_reaction_idx = force_reaction

    if add_homeostatic_demand is not None:
        # assert add_homeostatic_demand is a set
        assert isinstance(add_homeostatic_demand, list)

        for met in add_homeostatic_demand:
            homeostatic[met] = 100
            homeostatic_counts[met] = 1

    # Solve NetworkFlowModel
    model = NetworkFlowModel(
            stoich_arr=S_new,
            metabolites=metabolites,
            reactions=reaction_names,
            homeostatic_metabolites=metabolism.homeostatic_metabolites,
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
    model.set_up_exchanges(exchanges=exchange_molecules, uptakes=uptake)
    solution: FlowResult = model.solve(
            homeostatic_concs=homeostatic_counts, # in conc
            homeostatic_dm_targets=np.array(list(dict(homeostatic).values())), # *10^7
            maintenance_target=maintenance, # *10^6 ish
            kinetic_targets=np.array(list(dict(kinetic).values())), # *10^6 ish
            # binary_kinetic_idx=binary_kinetic_idx, #7646
            binary_kinetic_idx=None,
            force_flow_idx=force_reaction_idx,
            objective_weights=objective_weights, #same
            upper_flux_bound= 1000000000, # increase to 10^9 because notebook runs FlowResult using Counts, WC runs using conc.,
            kinetic_constraint=kinetic_constraint_dict,
            kinetic_adjustment=0.8,
            solver=solver_choice) #SCS. ECOS, MOSEK
    return solution.objective, solution.velocities, reaction_names, S_new, solution.kinetic_term, solution.homeostatic_term, solution.secretion_term, solution.efficiency_term

In [22]:
objective_weights = {'secretion': 0.0000001, 'efficiency': 0.000001, 'kinetics': 0.00001, 'homeostatic': 1}
oofv, solution_flux, _, _, _, _, _, _ = test_NetworkFlowModel(
                                            objective_weights
)
oofv

np.float64(169058.93879214922)

# Sweep across homeostatic weight to see how changes in homeostatic weight affects kinetic and homeostatic terms in the objective

In [24]:
objective_weights = {'secretion': 0.0000001, 'efficiency': 0.000001, 'kinetics': 0.00001, 'homeostatic': 1}
homeostatic_sweep = np.logspace(-5, 0, 30)

homeostatic_arr = []
kinetic_arr = []
secretion_arr = []
efficiency_arr = []
for sweep_num in homeostatic_sweep:
    objective_weights['homeostatic'] = sweep_num
    oofv, _, _, _, kinetic_term, homeostatic_term, secretion_term, efficiency_term= test_NetworkFlowModel(objective_weights)

    homeostatic_arr.append(homeostatic_term)
    kinetic_arr.append(kinetic_term * objective_weights['kinetics'])
    secretion_arr.append(secretion_term * objective_weights['secretion'])
    efficiency_arr.append(efficiency_term * objective_weights['efficiency'])

In [20]:
from plotly import graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=homeostatic_sweep,
    y=homeostatic_arr,
    mode='lines+markers',
    name='Homeostatic term',
    hovertemplate='homeostatic weight=%{x:.2e}<br>homeostatic term=%{y:.4g}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=homeostatic_sweep,
    y=homeostatic_arr*homeostatic_sweep,
    mode='lines+markers',
    name='Homeostatic term * weight',
    hovertemplate='homeostatic weight=%{x:.2e}<br>homeostatic term=%{y:.4g}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=homeostatic_sweep,
    y=kinetic_arr,
    mode='lines+markers',
    name='Kinetic term * weight',
    hovertemplate='homeostatic weight=%{x:.2e}<br>kinetic term=%{y:.4g}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=homeostatic_sweep,
    y=secretion_arr ,
    mode='lines+markers',
    name='Secretion term * weight',
    hovertemplate='homeostatic weight=%{x:.2e}<br>kinetic term=%{y:.4g}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=homeostatic_sweep,
    y=efficiency_arr,
    mode='lines+markers',
    name='Efficiency term * weight',
    hovertemplate='homeostatic weight=%{x:.2e}<br>kinetic term=%{y:.4g}<extra></extra>'
))

fig.update_layout(
    title='Trade-off vs. Homeostatic Weight',
    xaxis=dict(title='Homeostatic weight (λ_H)', type='log', tickformat='.0e'),
    yaxis=dict(type='log', tickformat='.0e'),
    template='plotly_white',
    legend=dict(title='Terms')
)

fig.show()

In [25]:
from plotly import graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=homeostatic_sweep,
    y=homeostatic_arr,
    mode='lines+markers',
    name='Homeostatic term',
    hovertemplate='homeostatic weight=%{x:.2e}<br>homeostatic term=%{y:.4g}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=homeostatic_sweep,
    y=homeostatic_arr*homeostatic_sweep,
    mode='lines+markers',
    name='Homeostatic term * weight',
    hovertemplate='homeostatic weight=%{x:.2e}<br>homeostatic term=%{y:.4g}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=homeostatic_sweep,
    y=kinetic_arr,
    mode='lines+markers',
    name='Kinetic term * weight',
    hovertemplate='homeostatic weight=%{x:.2e}<br>kinetic term=%{y:.4g}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=homeostatic_sweep,
    y=secretion_arr ,
    mode='lines+markers',
    name='Secretion term * weight',
    hovertemplate='homeostatic weight=%{x:.2e}<br>kinetic term=%{y:.4g}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=homeostatic_sweep,
    y=efficiency_arr,
    mode='lines+markers',
    name='Efficiency term * weight',
    hovertemplate='homeostatic weight=%{x:.2e}<br>kinetic term=%{y:.4g}<extra></extra>'
))

fig.update_layout(
    title='Trade-off vs. Homeostatic Weight',
    xaxis=dict(title='Homeostatic weight (λ_H)', type='log', tickformat='.0e'),
    yaxis=dict(type='log', tickformat='.0e'),
    template='plotly_white',
    legend=dict(title='Terms')
)

fig.show()

In [27]:
np.array(secretion_arr)/objective_weights['secretion']

array([1.16426351e+07, 1.13277004e+07, 1.06636114e+07, 1.03952692e+07,
       1.02083455e+07, 9.66685140e+06, 7.73389149e+06, 6.37118132e+06,
       6.88457318e+06, 7.08709372e+06, 7.07354830e+06, 7.06708630e+06,
       7.02478675e+06, 6.94893778e+06, 6.77438624e+06, 6.03007877e+06,
       6.02373997e+06, 1.95375097e+07, 3.77442828e+07, 3.77442828e+07,
       6.57509914e+07, 6.57509914e+07, 1.08293666e+08, 1.08293666e+08,
       1.08293666e+08, 1.08293666e+08, 1.08293666e+08, 1.08293666e+08,
       1.08293666e+08, 1.08293666e+08])

In [44]:
# --- 2) OPTIONAL: Pareto view in objective space (homeostatic vs kinetic) ---
fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=homeostatic_arr,
    y=kinetic_arr,
    mode='markers',
    name='Sweep path',
    hovertemplate='homeostatic=%{x:.4g}<br>kinetic=%{y:.4g}<extra></extra>'
))
fig2.update_layout(
    title='Objective Space (Homeostatic vs. Kinetic)',
    template='plotly_white',
    width=800, height=450,
    xaxis=dict(title='homeostatic term',
               gridcolor='lightgray',  # Set x-axis grid color
               type="log", tickformat='.0e'),
    yaxis=dict(
        title='Kinetic term',
        gridcolor='lightgray',  # Set y-axis grid color
        type="log", tickformat='.0e'
        ),
)
fig2.show()

In [12]:
# print list of homeostatic metabolite
metabolism.homeostatic_metabolites

array(['2-3-DIHYDROXYBENZOATE[c]', '2-KETOGLUTARATE[c]', '2-PG[c]',
       '2K-4CH3-PENTANOATE[c]', '4-AMINO-BUTYRATE[c]',
       '4-hydroxybenzoate[c]', 'ACETOACETYL-COA[c]', 'ACETYL-COA[c]',
       'ACETYL-P[c]', 'ADENINE[c]', 'ADENOSINE[c]', 'ADP-D-GLUCOSE[c]',
       'ADP[c]', 'AMP[c]', 'ANTHRANILATE[c]', 'APS[c]', 'ARG[c]',
       'ASN[c]', 'ATP[c]', 'BIOTIN[c]', 'CA+2[c]', 'CAMP[c]',
       'CARBAMYUL-L-ASPARTATE[c]', 'CARBON-DIOXIDE[c]', 'CDP[c]',
       'CHORISMATE[c]', 'CIS-ACONITATE[c]', 'CIT[c]', 'CL-[c]', 'CMP[c]',
       'CO+2[c]', 'CO-A[c]', 'CPD-12115[c]', 'CPD-12261[p]',
       'CPD-12575[c]', 'CPD-12819[c]', 'CPD-12824[c]', 'CPD-13469[c]',
       'CPD-2961[c]', 'CPD-8260[c]', 'CPD-9956[c]', 'CPD0-939[c]',
       'CTP[c]', 'CYS[c]', 'CYTIDINE[c]', 'CYTOSINE[c]', 'D-ALA-D-ALA[c]',
       'D-SEDOHEPTULOSE-7-P[c]', 'DAMP[c]', 'DATP[c]', 'DCTP[c]',
       'DEOXY-RIBOSE-5P[c]', 'DEOXYADENOSINE[c]', 'DEOXYGUANOSINE[c]',
       'DGMP[c]', 'DGTP[c]', 'DI-H-OROTATE[c]',
       '

In [30]:
metabolism.exchange_molecules

{'3-HYDROXYPHENYL-PROPIONATE[e]',
 '3-PHENYLPROPIONATE[e]',
 'ACET[p]',
 'AMMONIUM[c]',
 'BETAINE[p]',
 'BUTANAL[c]',
 'CA+2[p]',
 'CARBON-DIOXIDE[p]',
 'CARBON-MONOXIDE[p]',
 'CH33ADO[c]',
 'CL-[p]',
 'CO+2[p]',
 'CPD-10774[p]',
 'CPD-10797[e]',
 'CPD-108[p]',
 'CPD-1772[c]',
 'CPD-239[p]',
 'CPD-534[p]',
 'CPD-560[p]',
 'CPD-564[c]',
 'CPD-674[e]',
 'CPD0-1083[e]',
 'CPD0-1202[e]',
 'CPD0-2167[c]',
 'D-ALANINE[p]',
 'D-GALACTARATE[e]',
 'D-GLUCARATE[e]',
 'D-LACTATE[p]',
 'ETOH[p]',
 'FE+2[p]',
 'FE+3[p]',
 'FORMATE[p]',
 'GLC-1-P[e]',
 'GLC[p]',
 'GLYCEROL[p]',
 'GLYCOLALDEHYDE[c]',
 'GLYCOLLATE[c]',
 'HCN[e]',
 'HYDROGEN-MOLECULE[c]',
 'HYPOXANTHINE[p]',
 'INDOLE[p]',
 'K+[p]',
 'L-SELENOCYSTEINE[c]',
 'METOH[p]',
 'MG+2[p]',
 'MI-PENTAKISPHOSPHATE[p]',
 'MN+2[p]',
 'NA+[p]',
 'NI+2[p]',
 'OXAMATE[e]',
 'OXYGEN-MOLECULE[p]',
 'PROTON[p]',
 'Pi[p]',
 'S-ADENOSYL-4-METHYLTHIO-2-OXOBUTANOATE[c]',
 'S-ALLANTOIN[e]',
 'S2O3[e]',
 'SUC[p]',
 'SULFATE[p]',
 'SUPER-OXIDE[e]',
 'TARTRATE[p]

['PMPOXI-RXN', 'PNPOXI-RXN']

# Sweep across kinetic weights (simulate knockdown studies) with fixed excahnges (avoid infeasible increase in nutrient uptake to supply sufficient homeostatic need)

In [202]:
def solve_with_fixed_exchanges(
    model,
    fixed_exchanges: Optional[np.ndarray] = None,   # pass baseline e here, or None to not fix exchange flux
    *,
    homeostatic_concs: Iterable[float],
    homeostatic_dm_targets: Iterable[float],
    maintenance_target: float = maintenance,
    kinetic_targets: Optional[Iterable[float]] = None,
    binary_kinetic_idx: Optional[Iterable[int]] = None,
    force_flow_idx: Optional[Iterable[int]] = None,
    objective_weights: Optional[Mapping[str, float]] = None,
    upper_flux_bound: float = 1000000000.0,
    solver=cp.GLOP,   # keep whatever you use in your environment
):
    """
    Rebuilds the same optimization as NetworkFlowModel.solve but allows pinning exchanges.
    Returns a FlowResult (same as vEcoli).
    """
    objective_weights = cast(Mapping[str, float], objective_weights)

    # Variables
    v = cp.Variable(model.n_orig_rxns)
    e = cp.Variable(model.n_exch_rxns)
    dm = model.S_orig @ v + model.S_exch @ e
    exch = model.S_exch @ e

    # Constraints
    constr = []
    constr.append(dm[model.intermediates_idx] == 0)

    if model.maintenance_idx is not None:
        constr.append(v[model.maintenance_idx] == maintenance_target)

    if binary_kinetic_idx is not None:
        constr.append(v[binary_kinetic_idx] == 0)

    constr.extend([v >= 0, v <= upper_flux_bound, e >= 0, e <= upper_flux_bound])

    # NEW COMPARED TO vECOLI: keep exchanges fixed to the baseline sim
    if fixed_exchanges is not None:
        # assert(len(fixed_exchanges) == len(exch))
        constr.append(exch == fixed_exchanges)
    else:
        raise ValueError("No exchange flux to pin")

    # Objective
    loss = 0
    loss += cp.norm1((dm[model.homeostatic_idx] - homeostatic_dm_targets) / homeostatic_concs)

    if "secretion" in objective_weights:
        loss += objective_weights["secretion"] * cp.sum(e[model.secretion_idx])
    if "efficiency" in objective_weights:
        loss += objective_weights["efficiency"] * cp.sum(v)
    if "kinetics" in objective_weights:
        loss += objective_weights["kinetics"] * cp.norm1(v[model.kinetic_rxn_idx] - kinetic_targets)

    prob = cp.Problem(cp.Minimize(loss), constr)
    prob.solve(solver=solver, verbose=False)
    if prob.status != "optimal":
        raise ValueError(f"Fixed-exchange solve did not converge. Status: {prob.status}")

    # Build a FlowResult without depending on class constructor
    class _FlowResult:
        def __init__(self, velocities, dm_dt, exchange_rxn, all_met_exchange, objective):
            self.velocities = velocities
            self.dm_dt = dm_dt
            self.exchange_rxn = exchange_rxn
            self.all_met_exchange = all_met_exchange
            self.objective = objective

    return _FlowResult(
        velocities=np.array(v.value),
        dm_dt=np.array(dm.value),
        exchange_rxn=np.array(e.value), #temp
        all_met_exchange=np.array(exch.value),
        objective=prob.value,
    )

In [220]:
def sweep_kinetics_with_pinned_exchanges(
    model,
    *,
    base_kwargs: dict,
    kinetic_targets_list: list[np.ndarray],
):
    """
    1) Run baseline using model.solve(**base_kwargs) to get baseline exchanges.
    2) For each kinetic target vector in kinetic_targets_list, re-solve with exchanges pinned.
    3) Return a tidy DataFrame of dm/dt for homeostatic metabolites plus objective values.
    """
    # --- 1) Baseline run (no fixed exchanges) ---
    baseline_res = model.solve(**base_kwargs)
    exch0 = baseline_res.exchanges  # net exchange per metabolite

    # pass exch0 at baseline sim to be the fixed exchanges
    fixed_vec = np.array(exch0).astype(float)

    # --- 2) Re-solve for each kinetic target ---
    rows = []
    df_kinetic = []
    home_idx = model.homeostatic_idx
    home_mets = [model.mets[i] for i in home_idx]

    for j, kt in enumerate(kinetic_targets_list):
        kwargs_w_kt = {**base_kwargs, "kinetic_targets": kt}
        res = solve_with_fixed_exchanges(
            model,
            fixed_exchanges=fixed_vec,
            **kwargs_w_kt
        )

        # Pull homeostatic dm/dt and objective
        velocities = res.velocities
        dm_home = res.dm_dt[home_idx]
        row = {"sweep_id": j, "objective": res.objective}

        row.update({f"dm_dt[{met}]": dm_home[i] for i, met in enumerate(home_mets)})

        rows.append(row)

        col = velocities
        df_kinetic.append(col)

        # quick sanity check that exchanges truly stayed fixed
        # (checks S_exch @ e)
        tol = 1e-6
        assert np.allclose(res.all_met_exchange, exch0, atol=tol)

    df = pd.DataFrame(rows).set_index("sweep_id")
    df_kinetic = pd.DataFrame(df_kinetic, columns=reaction_names).set_index(pd.Index(range(len(kinetic_targets_list)), name="sweep_id"))
    return baseline_res, df, df_kinetic


In [230]:
# --- prepare base kwargs used by your original .solve() call ---
base_kwargs = dict(
    homeostatic_concs=homeostatic_count.copy() * metabolism.counts_to_molar.asNumber(),
    homeostatic_dm_targets=np.array(list(dict(homeostatic).values())),    # same length/order as above
    maintenance_target=maintenance,
    kinetic_targets=np.array(list(dict(kinetic).values())),           # baseline target, len = len(model.kinetic_rxn_idx)
    binary_kinetic_idx=None,
    force_flow_idx=None,
    objective_weights={'secretion': 0.001, 'efficiency': 0.000001, 'kinetics': 100},
    upper_flux_bound= 1000000000,
    solver=cp.GLOP,
)

# --- make a list of kinetic target vectors to try ---
baseline_kt = np.asarray(base_kwargs["kinetic_targets"], dtype=float)
scales = [0, 0.1, 0.5, 1.0]
kt_list = [s * baseline_kt for s in scales]

# --- run sweep with exchanges pinned to the baseline solution ---
model = metabolism.network_flow_model
baseline_res, sweep_df,  df_kinetic = sweep_kinetics_with_pinned_exchanges(
    model,
    base_kwargs=base_kwargs,
    kinetic_targets_list=kt_list,
)

# Inspect results
from IPython.display import display
print("Baseline objective:", baseline_res.objective)
display(sweep_df)


Baseline objective: 285904304.073986


,objective,dm_dt[2-3-DIHYDROXYBENZOATE[c]],dm_dt[2-KETOGLUTARATE[c]],dm_dt[2-PG[c]],dm_dt[2K-4CH3-PENTANOATE[c]],dm_dt[4-AMINO-BUTYRATE[c]],dm_dt[4-hydroxybenzoate[c]],dm_dt[ACETOACETYL-COA[c]],dm_dt[ACETYL-COA[c]],dm_dt[ACETYL-P[c]],...,dm_dt[ZN+2[p]],dm_dt[MN+2[p]],dm_dt[NI+2[p]],dm_dt[CO+2[p]],dm_dt[FE+2[p]],dm_dt[NA+[p]],dm_dt[OXYGEN-MOLECULE[p]],dm_dt[FE+3[p]],dm_dt[CA+2[p]],dm_dt[Pi[p]]
sweep_id,,,,,,,,,,,,,,,,,,,,,
0,7.259843e+06,0.000000,48.1425,12.5225,18.8225,41.46,0.00000,4.2175,-92.30500,145.93,...,13.64,13.64,13.64,13.64,13.64,13.64,13.72,13.7125,13.7125,13.6675
1,3.440122e+07,-333.623125,48.1425,12.5225,18.8225,41.46,7.12000,4.2175,81235.14325,145.93,...,13.64,13.64,13.64,13.64,13.64,13.64,13.72,13.7125,13.7125,13.6675
2,1.455937e+08,-1644.587500,48.1425,12.5225,18.8225,41.46,7.12000,4.2175,406724.93625,145.93,...,13.64,13.64,13.64,13.64,13.64,13.64,13.72,13.7125,13.7125,13.6675
3,2.859043e+08,-3289.175000,48.1425,12.5225,29768.0950,41.46,24.14375,4.2175,813587.17750,145.93,...,13.64,13.64,13.64,13.64,13.64,13.64,13.72,13.7125,13.7125,13.6675


In [228]:
display(df_kinetic[kinetic_reaction_ids])

,1.1.1.39-RXN,1.1.1.83-RXN,1.13.11.16-RXN,2.1.1.79-RXN-CPD-18361/S-ADENOSYLMETHIONINE//CPD-18373/ADENOSYL-HOMO-CYS/PROTON.67.,2.1.1.79-RXN-CPD-18362/S-ADENOSYLMETHIONINE//CPD-18406/ADENOSYL-HOMO-CYS/PROTON.67.,2.1.1.79-RXN-CPD-18367/S-ADENOSYLMETHIONINE//CPD-18371/ADENOSYL-HOMO-CYS/PROTON.67.,2.1.1.79-RXN-CPD-18369/S-ADENOSYLMETHIONINE//CPD-18372/ADENOSYL-HOMO-CYS/PROTON.67.,2.1.1.79-RXN-CPD-18392/S-ADENOSYLMETHIONINE//CPD-18405/ADENOSYL-HOMO-CYS/PROTON.67.,2.1.1.79-RXN-CPD-18403/S-ADENOSYLMETHIONINE//CPD-18404/ADENOSYL-HOMO-CYS/PROTON.67.,2.3.1.157-RXN,...,UDPGLUCEPIM-RXN,UDPGLUCEPIM-RXN (reverse),UDPHYDROXYMYRGLUCOSAMNACETYLTRANS-RXN,UDPNACETYLGLUCOSAMACYLTRANS-RXN,UDPNACETYLGLUCOSAMENOLPYRTRANS-RXN,UGD-RXN,UNDECAPRENYL-DIPHOSPHATASE-RXN[CCO-CYTOSOL]-UNDECAPRENYL-DIPHOSPHATE/WATER//CPD-9646/Pi/PROTON.64.__PGPPHOSPHAB-MONOMER,URA-PHOSPH-RXN,UROGENIIISYN-RXN,URPHOS-RXN__URPHOS-CPLX
sweep_id,,,,,,,,,,,,,,,,,,,,,
0,-0.00,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.00000,...,-0.0,-0.00000,-0.0000,-0.0000,582.535,-0.0,-0.0000,-0.0,348.635312,-0.00000
1,1907.51,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,11467.40625,...,452.2,323.72175,256.9565,256.9565,582.535,6.5,1.8425,132.7,348.635312,3668.17925
2,9537.55,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,57337.03125,...,2261.0,1618.60875,1284.7825,1284.7825,582.535,32.5,9.2125,663.5,601.634844,18340.89625
3,19075.10,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,114674.06250,...,4522.0,3237.21750,2569.5650,2569.5650,582.535,65.0,18.4250,1327.0,1203.269688,36681.79250


In [178]:
baseline_res = solve_with_fixed_exchanges(model,**base_kwargs)
e0 = baseline_res.exchange_metabolite             # vector length = n_exch_rxns

In [180]:
e0

array([-0.00000000e+00,  3.40178549e+07, -0.00000000e+00,  1.26450000e+01,
        9.54274500e+03, -0.00000000e+00,  2.80250000e+00, -0.00000000e+00,
        3.08950000e+01, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        2.12308010e+07, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        4.20022500e+02, -0.00000000e+00,  3.66725000e+01, -0.00000000e+00,
       -0.00000000e+00,  1.37125000e+01, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  1.53450000e+01, -0.00000000e+00, -0.00000000e+00,
        3.80296481e+06, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  9.96869850e+04, -0.00000000e+00,  1.26450000e+01,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  1.33300000e+01,
       -0.00000000e+00, -0.00000000e+00,  2.37673577e+06, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00,  6.04075000e+01, -0.00000000e+00,
        3.54600000e+01, -

In [181]:
exch0 = (model.S_exch @ e0)

In [164]:
model.S_exch

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 87 stored elements and shape (6078, 87)>

In [166]:
len(e0)

6078

In [183]:
metabolism.network_flow_model.n_exch_rxns

87

In [184]:
metabolism.network_flow_model.S_exch

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 87 stored elements and shape (6078, 87)>

In [185]:
len(exchange_molecules)

68

In [186]:
len(allowed_exchange_uptake)

19

In [188]:
model.n_mets

6078

In [191]:
len(baseline_res.exchange_metabolite != 0)

87

In [233]:
df = pd.DataFrame(kinetic_target_essential)
df

,0
2.3.1.157-RXN,114674.0625
2.7.7.60-RXN,2587.9500
2.8.1.6-RXN,226.0000
ACETYLORNDEACET-RXN,88512.4450
ADENYLOSUCCINATE-SYNTHASE-RXN,4950.0825
...,...
SERINE-O-ACETTRAN-RXN__CPLX0-237,0.0000
TRIPHOSPHATASE-RXN[CCO-CYTOSOL]-P3I/WATER//PPI/Pi/PROTON.38.__CPLX0-243,36608.3225
UDP-NACMUR-ALA-LIG-RXN,1137.0100
UDPHYDROXYMYRGLUCOSAMNACETYLTRANS-RXN,2569.5650


In [234]:
fba.keys()

dict_keys(['solution_fluxes', 'solution_dmdt', 'solution_residuals', 'time_per_step', 'estimated_fluxes', 'estimated_homeostatic_dmdt', 'homeostatic_metabolite_counts', 'target_homeostatic_dmdt', 'estimated_exchange_dmdt', 'estimated_intermediate_dmdt', 'target_kinetic_fluxes', 'target_kinetic_bounds', 'reaction_catalyst_counts', 'maintenance_target'])

In [250]:
fba.keys()

dict_keys(['solution_fluxes', 'solution_dmdt', 'solution_residuals', 'time_per_step', 'estimated_fluxes', 'estimated_homeostatic_dmdt', 'homeostatic_metabolite_counts', 'target_homeostatic_dmdt', 'estimated_exchange_dmdt', 'estimated_intermediate_dmdt', 'target_kinetic_fluxes', 'target_kinetic_bounds', 'reaction_catalyst_counts', 'maintenance_target'])

In [251]:
solution_df = pd.DataFrame(np.array(fba['estimated_fluxes'][1:]).mean(axis=0), index=reaction_names)
solution_df_essential = solution_df.loc[kinetic_target_essential.index]

In [252]:
solution_df_essential

,0
2.3.1.157-RXN,687.0350
2.7.7.60-RXN,371.1275
2.8.1.6-RXN,2.8025
ACETYLORNDEACET-RXN,14341.5375
ADENYLOSUCCINATE-SYNTHASE-RXN,17669.5700
...,...
SERINE-O-ACETTRAN-RXN__CPLX0-237,0.0000
TRIPHOSPHATASE-RXN[CCO-CYTOSOL]-P3I/WATER//PPI/Pi/PROTON.38.__CPLX0-243,0.0000
UDP-NACMUR-ALA-LIG-RXN,0.0000
UDPHYDROXYMYRGLUCOSAMNACETYLTRANS-RXN,0.0000


In [255]:
merged_df = pd.merge(df, solution_df_essential, left_index=True, right_index=True)
merged_df.columns = ['kinetic_target_flux', 'sim_flux']

In [256]:
merged_df

,kinetic_target_flux,sim_flux
2.3.1.157-RXN,114674.0625,687.0350
2.7.7.60-RXN,2587.9500,371.1275
2.8.1.6-RXN,226.0000,2.8025
ACETYLORNDEACET-RXN,88512.4450,14341.5375
ADENYLOSUCCINATE-SYNTHASE-RXN,4950.0825,17669.5700
...,...,...
SERINE-O-ACETTRAN-RXN__CPLX0-237,0.0000,0.0000
TRIPHOSPHATASE-RXN[CCO-CYTOSOL]-P3I/WATER//PPI/Pi/PROTON.38.__CPLX0-243,36608.3225,0.0000
UDP-NACMUR-ALA-LIG-RXN,1137.0100,0.0000
UDPHYDROXYMYRGLUCOSAMNACETYLTRANS-RXN,2569.5650,0.0000


In [260]:
sum(merged_df['sim_flux'] > merged_df['kinetic_target_flux'])

13

In [258]:
len(merged_df['sim_flux'])

70

In [1]:
#TODO: add kinetic constraints where we cap the kinetic flux to 80% simulated flux.

In [18]:
kinetic_term * objective_weights['kinetics']

np.float64(1094.6883898517797)

In [20]:
homeostatic_term * objective_weights['homeostatic']

np.float64(1051.7168060665017)

In [17]:
objective_weights

{'secretion': 0.001, 'efficiency': 1e-06, 'kinetics': 1e-05, 'homeostatic': 1}

# scratch

In [ ]:
objective_weights = {'secretion': 0.001, 'efficiency': 0.000001, 'kinetics': 0.00001, 'homeostatic': 1}
add_kinetic = {r:kinetic[r]*0.2 for r in kinetic_reaction_id_essential} # 80% knockdown
add_reaction = {}

oofv, solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic, kinetic_term, homeostatic_term = test_NetworkFlowModel(
                                            objective_weights,
                                            add_kinetic=add_kinetic,
)

sim_flux = pd.DataFrame({
    'flux': solution_flux,
    'is_new': [
        'New Reactions' if id in fba_new_reaction_ids
        else 'TEMP' if id in add_reaction.keys()
        else 'Old Reactions'
            for id in test_reaction_names
    ]
}, index=test_reaction_names)


In [ ]:

objective_weights = {'secretion': 0.001, 'efficiency': 0.000001, 'kinetics': 0.00001}
oofv_stored = {}
for catalyst in kinetic_catalyst_overlapped:
    reactions = get_keys(kinetic_reaction_catalysts_essential, catalyst)
    add_kinetic = {r:kinetic[r]*0.2 for r in reactions} # 80% knockdown of catalyst's reactions
    oofv, solution_flux, test_reaction_names, S_new, _, test_kinetic = test_NetworkFlowModel(
                                                objective_weights,
                                                add_kinetic=add_kinetic,
    )
    oofv_stored[catalyst] = oofv
oofv, solution_flux, test_reaction_names, S_new, _, test_kinetic = test_NetworkFlowModel(
                                                objective_weights)
oofv_stored['default'] = oofv